<a href="https://colab.research.google.com/github/HYLee810/projectmanager/blob/main/2_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D_%EC%98%88%EC%A0%9C_%EB%B9%84%EC%A0%95%ED%98%95%EB%8D%B0%EC%9D%B4%ED%84%B0_%EA%B5%AC%EA%B8%80%EC%9C%84%EC%B9%98%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%8B%9C%EA%B0%81%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# (나의) 구글 Location History 데이터 지도에 표시하기

-------------------------------

### 지도 라이브러리

In [1]:
!pip install folium

In [2]:
import folium

folium.__version__

'0.14.0'

In [3]:
import folium

#------------------
# 중심위치 [위도,경도]=[Latitude, longitude]
#------------------
m = folium.Map(location = [37.5393556, 127.0667081],
               zoom_start=16)

# 마커 표시하기
folium.Marker(location = [37.5393556, 127.0667081],
              popup = '동부여성발전센터',
              icon=folium.Icon(color='red', icon='star')
).add_to(m)
m

## [실습] (나의) 구글 Location History 데이터 지도에 표시하기


### 1. 구글 Location History 데이터 다운로드
- 구글 크롬 브라우저에 로그인 된 상태에서
- https://takeout.google.com/settings/takeout 사이트로 이동하기
- 모두 체크를 해제시키고 **[위치 기록]** 만 체크표시하고
- takeout~.zip 파일이 다운로드 된다.

### 2. 필요한 라이브러리 설치하기
설치시 오류가 발생하면 윈도우 명령프롬프트창에서 timezonefinder 설치를 진행해 본다.

In [4]:
import numpy as np

np.__version__

'1.23.5'

In [5]:
!pip install timezonefinder

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 16.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 62.4 MB/s eta 0:00:00
  Created wheel for timezonefinder: filename=timezonefinder-6.2.0-cp310-cp310-manylinux_2_35_x86_64.whl size=46900632 sha256=1ddd51ad0ecb9ad5b04ceac78ed28793b72745ef11bf2b2abaae5fcd5608e2eb
  Stored in directory: /root/.cache/pip/wheels/17/19/d4/ae94459b7f74f7e8f171862d1c08adedf9b7c76ddfc514a620
Successfully built timezonefinder


In [6]:
# 경도, 위도값으로 시간대 정보 가져오기
from timezonefinder import TimezoneFinder

tzf= TimezoneFinder()
tz = tzf.timezone_at(lng=128, lat=38)  # 'Asia/Seoul'
#tz = tzf.timezone_at(lng=13.358, lat=52.5061)  # 'Europe/Berlin'
tz

'Asia/Seoul'

### 3.나의 Local History데이터 정보를 딕셔너리타입으로 변환하기

In [7]:
from zipfile import ZipFile
import pandas as pd
import json
from timezonefinder import TimezoneFinder

In [8]:
# Google Location History takeout file
in_file = "data/takeout-20230905T023235Z-001.zip"
out_file = "data/place_visits.csv" # 전처리 끝난 파일 저장

# store all places into this array
place_visits = []

with ZipFile(in_file) as myzip:
    for file in myzip.filelist[:]:
        filename = file.filename

        if "Semantic Location History" in filename:
            # process all files in "Semantic Location History" directory
            history_json = json.load(myzip.open(filename))

            for timeline_object in history_json["timelineObjects"]:

                if "placeVisit" in timeline_object:
                    place_visit_json = timeline_object["placeVisit"]

                    # 위도.경도 값 없으면 skip
                    if not "location" in place_visit_json or not "latitudeE7" in place_visit_json["location"]:
                        continue

                    place_visit = {
                        "placeId": place_visit_json["location"]["placeId"],
                        "locationConfidence": place_visit_json["location"]["locationConfidence"],
                        "startTimestamp": place_visit_json["duration"]["startTimestamp"],
                        "endTimestamp": place_visit_json["duration"]["endTimestamp"],
                        "placeVisitImportance": place_visit_json["placeVisitImportance"],
                        "placeVisitType": place_visit_json["placeVisitType"],
                        "latitudeE7": place_visit_json["location"]["latitudeE7"],
                        "longitudeE7": place_visit_json["location"]["longitudeE7"],
                    }

                    for optional_field in ["centerLatE7", "centerLngE7"]:
                        if optional_field in place_visit_json:
                            place_visit[optional_field] = place_visit_json[optional_field]
                        else:
                            place_visit[optional_field] = None

                    for optional_field in ["name", "address"]:
                        if optional_field in place_visit_json["location"]:
                            place_visit[optional_field] = place_visit_json["location"][optional_field]
                        else:
                            place_visit[optional_field] = None


                    place_visits.append(place_visit)

print(f'레코드 총 개수: {len(place_visits)}')
place_visits[:3]

FileNotFoundError: ignored

### 4. DataFrame 형태로 변환하기

In [ ]:
# 1.DataFrame형태로 변환하기
place_visits_df = pd.DataFrame(place_visits)
place_visits_df.head(2)


# 2.데이터 전처리하기
# convert to datetime type
place_visits_df["startTimestamp"] = pd.to_datetime(place_visits_df["startTimestamp"])
place_visits_df["endTimestamp"] = pd.to_datetime(place_visits_df["endTimestamp"])


# get geo coordinates as float value
place_visits_df["latitude"] = place_visits_df.latitudeE7/10**7   #(1E7 = 1e7 = 1x10^7 = 10**7)
place_visits_df["longitude"] = place_visits_df.longitudeE7/1e7
place_visits_df["centerLat"] = place_visits_df.centerLatE7/1E7
place_visits_df["centerLng"] = place_visits_df.centerLngE7/1E7

# add timezone based on geo coordinates
tf = TimezoneFinder()
place_visits_df["timezone"] = place_visits_df.apply(lambda row: tf.timezone_at(lng=row.longitude, lat=row.latitude), axis=1)

# convert UTC time to local timezone
place_visits_df['startTimestamp_local'] = place_visits_df.apply(lambda row: row.startTimestamp.tz_convert(row.timezone), axis=1)
place_visits_df['endTimestamp_local'] =place_visits_df.apply(lambda row: row.endTimestamp.tz_convert(row.timezone), axis=1)

# remove TZ info from datetime
place_visits_df['startTimestamp_local'] = pd.to_datetime(place_visits_df['startTimestamp_local'].apply(lambda x: x.replace(tzinfo=None)))
place_visits_df['endTimestamp_local'] = pd.to_datetime(place_visits_df['endTimestamp_local'].apply(lambda x: x.replace(tzinfo=None)))

# add datetime parts as a separate column to data frame
for datetime_type in [("year", lambda x: x.year), ("month", lambda x: x.month), ("day", lambda x: x.day), ("hour", lambda x: x.hour), ("minute", lambda x: x.minute), ("weekday", lambda x: x.weekday)]:
    for tztype in ["", "_local"]:
        place_visits_df[f"{datetime_type[0]}{tztype}"] = datetime_type[1](place_visits_df[f"startTimestamp{tztype}"].dt)


place_visits_df.drop(columns=["latitudeE7", "longitudeE7", "centerLatE7", "centerLngE7"], inplace=True)
place_visits_df["duration"] = place_visits_df.endTimestamp - place_visits_df.startTimestamp
place_visits_df["duration_minutes"] = place_visits_df.duration.dt.total_seconds()/60


# 3. 파일로 저장하기
place_visits_df.to_csv(in_file, index=False)


place_visits_df.head(2)

,placeId,locationConfidence,startTimestamp,endTimestamp,placeVisitImportance,placeVisitType,name,address,date,latitude,...,day,day_local,hour,hour_local,minute,minute_local,weekday,weekday_local,duration,duration_minutes
0,ChIJiWKXIGxhezURSdDzm8-jpb0,58.957363,2022-06-16 08:36:59.999000+00:00,2022-06-16 23:17:07.624000+00:00,MAIN,SINGLE_PLACE,501 광명역세권휴먼시아5단지,대한민국 광명시 소하2동,2022-06-16 08:36:59.999000+00:00,37.430855,...,16,16,8,17,36,36,3,3,0 days 14:40:07.625000,880.127083
1,ChIJ3Sey7a-ofDURam8Ne0dOt3A,28.873331,2022-06-17 00:02:27.492000+00:00,2022-06-17 03:22:09.877000+00:00,MAIN,SINGLE_PLACE,가천대학교 기숙사,대한민국 성남시 복정동,2022-06-17 00:02:27.492000+00:00,37.454329,...,17,17,0,9,2,2,4,4,0 days 03:19:42.385000,199.706417


### 5. (나의)Location History 지도에 표시하기

In [ ]:
import folium
import webbrowser
import pandas as pd

# 파일 불러오기
# df = pd.read_csv(file)
df = place_visits_df


center = [37.4306881, 126.8845086]
# m = folium.Map(location=center ,
#                   zoom_start=6,
#                   width=750,
#                   height=500)
m = folium.Map(location=center ,
                  zoom_start=6)

# df에서 위도&경도 정보 지도 Marker에 추가하기
folium.Marker(location=center,
                popup='우리집', icon=folium.Icon(color='red', icon='star'),
                color='tomato', radius = 50,
).add_to(m)


# 구글 Location History 마커로 표시하기
for i, row in df.iterrows():
    # 아이콘 색상 지정
    date = row['startTimestamp'].strftime('%Y-%m-%d')  # timestamp -> string타입으로 변환
    if date <= '2022-12-31':
        icon_color='green'
    elif date < '2023-07-01':
        icon_color='blue'
    else:
        icon_color='red'
    # 마커 추가하기
    folium.Marker(location= [row['latitude'], row['longitude']],
                  icon=folium.Icon(icon='cloud', color=icon_color),
                  popup=row['name']
    ).add_to(m)

m

#html 파일로 저장하여 불러오기
m.save('data/map_google_lh.html')
webbrowser.open('C:/python/projectmanager/data/map_google_lh.html')

True

--------------------

THE END